In [2]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
from scipy.io import loadmat
from scipy import signal
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit, cross_val_score, train_test_split, StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

from mne import Epochs, pick_types, events_from_annotations, EpochsArray, create_info
from mne.channels import make_standard_montage
from mne.io import concatenate_raws, read_raw_edf
from mne.datasets import eegbci
from mne.decoding import CSP
from mne.time_frequency import psd_array_welch, psd_array_multitaper, stft
import seaborn as sns
from tqdm import tqdm

In [5]:
# load batch_stft.npy and train_idx.npy
batch_stft = np.load('batch_stft.npy')
train_idx = np.load('train_idx.npy')
labels = np.load('labels.npy')

train_stft = batch_stft[train_idx]
train_labels = labels[train_idx]
welch_train = np.mean(train_stft, axis=3)



In [6]:
print(welch_train.shape)
print(train_labels.shape)

(3725, 64, 129)
(3725,)


In [7]:
# train logistic regression model

X_train = welch_train.reshape(welch_train.shape[0], -1)
X_test = np.mean(batch_stft[~train_idx], axis=3)
X_test = X_test.reshape(X_test.shape[0], -1)
y_test = labels[~train_idx]

# Fit a logistic regression model
clf = LogisticRegression()
clf.fit(X_train, train_labels)

# Predict the labels for the test set
y_pred = clf.predict(X_test)

# Compute the accuracy score
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)


Accuracy: 0.6381208053691275


/Users/anushmutyala/opt/miniconda3/envs/torch/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
